In [5]:
import logging
logging.basicConfig(filename='logger.log', level=logging.INFO)

In [6]:
import stanza
zh_nlp = stanza.Pipeline('zh', processors='tokenize,ner', use_gpu=False)
def ner(text):
    logging.info('len of text: {}'.format(len(text)))
    doc = zh_nlp(text)
    entities=doc.ents
    print('len of origin entities size:',len(entities))
    return [x.to_dict() for x in entities if len(x.to_dict()['text'])>1]

In [7]:
import os
data=[]
base_path='D:\Projects\金融事件抽取项目\金融事件抽取\所有爬虫语料'
def scan_dir(base_path):
    for web_name in os.listdir(base_path):
        logging.info("{} begin===".format(web_name))
        web_path=os.path.join(base_path, web_name)
        for k in os.listdir(web_path):# 关键词
            # print('current keyword:'+k)
            k_path=os.path.join(web_path, k)
            for file_name in os.listdir(k_path): # 文档集合
                final_path=os.path.join(k_path, file_name)
                with open(final_path,'r',encoding='utf-8', errors='ignore') as f:
                    content=f.read().strip()
                    if len(content)>0:
                        entities=ner(content)
                        data.append([web_name,k,content,entities])
                if len(data)>5:
                    print('test break===')
                    return
        print(web_name+'end=====')
scan_dir(base_path)
print('all files has been processed=====')

人民网产经begin=====
len of origin entities size: 105
len of origin entities size: 45
len of origin entities size: 94
len of origin entities size: 70
len of origin entities size: 68
len of origin entities size: 101
test break===
all files has been processed=====


In [8]:
print(len(data))
data[:2]

6


[['人民网产经',
  '信息泄露',
  '消费火正旺\xa0金融添把柴\n年轻人消费方式的改变和信用消费意识的增强，推动消费金融行业井喷式增长\n“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日本、缅甸和好几个欧洲国家，土耳其、以色列、厄瓜多尔的加拉帕戈斯群岛都在我未来的旅行清单上。”上海某律所的年轻律师吴辰伟对记者说，尽管自己收入还不错，但动辄两三万元的旅行花费也让他觉得有些头疼。所以今年“十一”假期和女友去塞班岛度假，吴辰伟在去哪儿旅行网预订时就选择了分期付款。“包含往返机票和五星级酒店住宿的7天塞班岛自由行，如果一次性付款，两个人总价是22100多元，我选择的是分12期付款，每期还1960元左右，这种付款方式为我减轻了用钱压力。不一定赚多少才能花多少，‘先消费后付款’很受我们年轻人欢迎。”吴辰伟说。\n80后、90后等年轻一代消费方式的改变和信用消费意识的增强，是近些年我国消费金融行业蓬勃发展的重要动力。来自山东临沂的刘明露今年24岁，在天津一家创业公司担任行政助理。“我平时喜欢看日本动漫和综艺，慢慢地对日语产生了兴趣。但是我收入不高，扣除房租和日常开销后，工资所剩有限，没办法一次性付清学日语的钱。”刘明露说，近两年她喜欢上了“先消费后付款”的模式。去年她用蚂蚁花呗在淘宝上分24期购买了一部华为P10手机，今年3月用招行信用卡刷了一台联想笔记本电脑。“这次我报名沪江网校的日语课用的是京东白条，分6期免息，每月还款510元。以后我还想学学商务英语，只要在自己的承受范围内且能及时还款，花未来的钱满足今天的愿望，我觉得是一笔划算的买卖。”\n银保监会近日下发通知，要求积极发展消费金融，适应多样化、多层次消费需求，提供和改进差异化金融产品与服务，支持发展消费信贷，创新金融服务方式，积极满足旅游、教育、文化、健康、养老等升级型消费的金融需求。中国人民大学重阳金融研究院高级研究员董希淼说，与之前的政策措施相比，“积极发展消费金融”是比较值得注意的信号。扩大内需是下半年经济工作的重要着力点，近段时间消费增速有所回落，应着力发挥金融对消费的促进和提升作用。\n苏宁金融研究院互联网金融研究中心主任薛洪言认为，正是近期的消费升级大大促进了消费金融的发展；反过来，消费金融的快速发展，也会进一步助推居民消费提质升级。尤其是从行业看，借

In [10]:
import pandas as pd
df=pd.DataFrame(data, columns=['src_web', 'keywords', 'text','entities'])
df['len_text']=df['text'].str.len()
df.head(2)

,src_web,keywords,text,entities,len_text
0,人民网产经,信息泄露,消费火正旺 金融添把柴\n年轻人消费方式的改变和信用消费意识的增强，推动消费金融行业井喷式增...,"[{'text': '韩国', 'type': 'GPE', 'start_char': 8...",3241
1,人民网产经,信息泄露,又一老人被骗280万元！这种诈骗来势凶猛 务必学会辨认\n广州市公安局9月2日发出紧急通报：...,"[{'text': '280万元', 'type': 'MONEY', 'start_cha...",1195


In [11]:
df['keywords'].value_counts()# 统计关键词文档数量

信息泄露    6
Name: keywords, dtype: int64

In [12]:
# 合并后缀为2的关键词
df['keywords']=df['keywords'].apply(lambda x: x[:-1] if x.endswith('2') else x)

In [13]:
df['keywords'].value_counts()

信息泄露    6
Name: keywords, dtype: int64

In [ ]:
# 关键词分布图
import seaborn as sns
sns.set_style('whitegrid',{'font.sans-serif':['simhei','Arial']})
sns.countplot(y=df['keywords'])

In [ ]:
df['len_text'].describe() # 文本长度分布

In [14]:
# df.to_csv('fin_articles.csv',encoding='utf-8',index=0)
df.to_csv('test_fin_articles.csv',encoding='utf-8',index=0)

In [15]:
df2=pd.read_csv('test_fin_articles.csv',encoding='utf8')
df2.head()

,src_web,keywords,text,entities,len_text
0,人民网产经,信息泄露,消费火正旺 金融添把柴\n年轻人消费方式的改变和信用消费意识的增强，推动消费金融行业井喷式增...,"[{'text': '韩国', 'type': 'GPE', 'start_char': 8...",3241
1,人民网产经,信息泄露,又一老人被骗280万元！这种诈骗来势凶猛 务必学会辨认\n广州市公安局9月2日发出紧急通报：...,"[{'text': '280万元', 'type': 'MONEY', 'start_cha...",1195
2,人民网产经,信息泄露,"月花两三千，过得像月入三五万？""租一族""崛起！但真的划算吗？\n原标题：月花两三千，过得像月...","[{'text': '三千', 'type': 'CARDINAL', 'start_cha...",2501
3,人民网产经,信息泄露,2020年我国数据总量全球占比将达20% 信息基础设施保护是大数据安全关键\n原标题：202...,"[{'text': '2020年', 'type': 'DATE', 'start_char...",1926
4,人民网产经,信息泄露,1秒钟能下载一部高清电影 5G来了！你准备好了吗\n原标题：5G来了！你准备好了吗\n5月1...,"[{'text': '1秒', 'type': 'TIME', 'start_char': ...",2030


In [ ]:
# 抽取实体